In [157]:
import soundfile, librosa, pickle, glob, os, parselmouth, pywt, statistics, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from parselmouth.praat import call
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC 
import numpy as geek
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# all emotions on RAVDESS dataset
allemotion = {
    "01": "angry",
    "02": "happy",
    "03": "neutral",
    "04": "sad"
}
# all gender on  dataset
allgender = {
    "01": "male",
    "02": "female"
}
# Emotions to observe
observed_emotions={
    "angry",
    "sad",
    "neutral",
    "happy"
}
# gender to observe
observed_gender={
    "male",
    "female"
}

In [122]:
#Extract features (mfcc, mel, ....) from a sound file
def extract_file_features(file_name, gender, **kwargs):  
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        sound = parselmouth.Sound(file_name) # read the sound
        
        
        # Spectral features (MFCC, Mel, LTAS)       
        # MFCC:
        mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=12).T, axis=0)
        # Mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        # LTAS
        spectrum = sound.to_spectrum()
        ltas = call(spectrum, "To Ltas (1-to-1)")
        ltas_mean = call(ltas, "Get mean", 0, 0, "dB") 
        ltas_sd = call(ltas, "Get standard deviation", 0, 0, "dB")
        ltas_max = call(ltas, "Get maximum", 0, 0, "parabolic")
        ltas_min = call(ltas, "Get minimum", 0, 0, "parabolic")
        ltas_range = ltas_max - ltas_min
        ltas_slope = call(ltas, "Get slope", 0, 1000, 1000, 4000.0, "dB")
        # Formants
        formant= sound.to_formant_burg()
        f1_mean = call(formant, "Get mean", 1, 0, 0, "Hertz") # get mean 1st Formant
        f2_mean = call(formant, "Get mean", 2, 0, 0, "Hertz") # get mean 2st Formant
        f3_mean = call(formant, "Get mean", 3, 0, 0, "Hertz") # get mean 3st Formant
        f1_sd = call(formant, "Get standard deviation", 1, 0, 0, "Hertz") # get standard deviation 1st Formant
        f2_sd = call(formant, "Get standard deviation", 2, 0, 0, "Hertz") # get standard deviation 2st Formant
        f3_sd = call(formant, "Get standard deviation", 3, 0, 0, "Hertz") # get standard deviation 3st Formant       

        # Prosodic features (pitch, intensity, formant, Jitter, Shimmer,,, HNR, Jitter, Shimmer)
        # Pitch
        if gender == "male":
            pitch = call(sound, "To Pitch", 0, 75, 300) #create a praat pitch object
        if gender == "female":
            pitch = call(sound, "To Pitch", 0, 100, 500) #create a praat pitch object
        
        #pitch = sound.to_pitch()
        pitch_mean = call(pitch, "Get mean", 0, 0, "Hertz") 
        pitch_sd = call(pitch, "Get standard deviation", 0 ,0, "Hertz") 
        pitch_max = call(pitch, "Get maximum", 0, 0, "Hertz", "Parabolic") 
        pitch_min = call(pitch, "Get minimum", 0, 0, "Hertz", "Parabolic")
        pitch_range = pitch_max - pitch_min
        
        # Intensity
        intensity = sound.to_intensity() 
        intensity_mean = call(intensity, "Get mean", 0, 0) 
        intensity_sd = call(intensity, "Get standard deviation", 0 ,0) 
        intensity_max = call(intensity, "Get maximum", 0, 0, "Parabolic")
        intensity_min = call(intensity, "Get minimum", 0, 0, "Parabolic") 
        intensity_range = intensity_max - intensity_min
        
        # HNR
        harmonicity = sound.to_harmonicity() #create a praat harmonicity object
        hnr = call(harmonicity, "Get mean", 0, 0) # get Harmonic to Noise Ratio
 
        #duration
        duration = sound.get_end_time()

        # Jitter and shimmer  
        pointProcess = call(sound, "To PointProcess (periodic, cc)", 75.0, 600.0)
        #jitter
        localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) # local Jitter
        localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3) # local-absolute Jitter
        rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3) # rap Jitter
        ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3) # ppq5 Jitter
        ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3) # ddp Jitter
        #Shimmer
        localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # local Shimmer
        localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # local-db Shimmer
        apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # apq3 Shimmer
        aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # aqpq5S himmer
        apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # apq11 Shimmer
        ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6) # dda Shimmer
      
   
        return mfcc, mel, ltas_mean, ltas_sd, ltas_max, ltas_min, ltas_range, ltas_slope, f1_mean, f1_sd, f2_mean, f2_sd, f3_mean, f3_sd, pitch_mean, pitch_sd, pitch_max, pitch_min, pitch_range, intensity_mean, intensity_sd, intensity_max, intensity_min, intensity_range, hnr, duration,localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [123]:
def runPCA(df, t):
    #Z-score the Jitter and Shimmer measurements
    if t=="jitter":
        features = ['local Jitter', 'local-absolute Jitter', 'rap Jitter', 'ppq5 Jitter', 'ddp Jitter']
    if t=="shimmer":
        features = ['local Shimmer', 'local-db Shimmer', 'apq3 Shimmer', 'apq5 Shimmer', 'apq11 Shimmer', 'dda Shimmer']
      
    # Separating out the features
    x = df.loc[:, features].values
    
    # Standardizing the features
    x = StandardScaler().fit_transform(x)   

    #PCA
    pca = PCA(n_components=1)
    principalComponents = pca.fit_transform(x)
    
    #PCA_list = principalComponents.tolist()
    if t=="jitter":
        principalDf = pd.DataFrame(data = principalComponents, columns = ['Jitter PCA'])
        pcalist = principalComponents.tolist()
    if t=="shimmer":
        principalDf = pd.DataFrame(data = principalComponents, columns = ['shimmer PCA'])
        pcalist = principalComponents.tolist()

    return principalDf, pcalist

In [124]:
# create lists to put the results
file_list, speaker_list, gender_list, emotion_list = [], [], [], []
jitterpca_list = []
shimmerpca_list = []
mfcc_list, mel_list = [], []
ltas_mean_list, ltas_sd_list, ltas_max_list, ltas_min_list, ltas_range_list, ltas_slope_list =  [], [], [], [], [], []
f1_mean_list, f1_sd_list, f2_mean_list, f2_sd_list, f3_mean_list, f3_sd_list =  [], [], [], [], [], []
pitch_mean_list, pitch_sd_list, pitch_max_list, pitch_min_list, pitch_range_list =  [], [], [], [], []
intensity_mean_list, intensity_sd_list, intensity_max_list, intensity_min_list, intensity_range_list =  [], [], [], [], []
hnr_list, duration_list =  [], []
localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list = [], [], [], [], []
localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list = [], [], [], [], [], []
  
# Load the data
def extract_all_features():
    for file in glob.glob("Data/NActor_*/*.wav"): #, recursive=True
        # get the base name of the audio file
        basename = os.path.basename(file)
        # get the emotion label
        emotion = allemotion[basename.split(" ")[0].split("-")[3]]
        # get the gender label
        gender = allgender[basename.split(" ")[0].split("-")[1]]
        # get the speaker label
        speaker = basename.split(" ")[0].split("-")[2]
    
        if emotion not in observed_emotions:
            continue
        if gender not in observed_gender:
            continue
            
            
        # extract speech features
        (mfcc, mel, ltas_mean, ltas_sd, ltas_max, ltas_min, ltas_range, ltas_slope, 
         f1_mean, f1_sd, f2_mean, f2_sd, f3_mean, f3_sd, pitch_mean, pitch_sd, pitch_max, pitch_min, pitch_range, 
         intensity_mean, intensity_sd, intensity_max, intensity_min, intensity_range, hnr, duration, localJitter, 
         localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, 
         apq11Shimmer, ddaShimmer) = extract_file_features(file, gender, mfcc=True, chroma=True, mel=True)   
        
        file_list.append(basename) # make an ID list
        speaker_list.append(speaker)
        gender_list.append(gender)
        emotion_list.append(emotion)
        mfcc_list.append(mfcc) # make a MFCCs list
        mel_list.append(mel) # make a Mel list
        ltas_mean_list.append(ltas_mean)
        ltas_sd_list.append(ltas_sd)
        ltas_max_list.append(ltas_max)
        ltas_min_list.append(ltas_min)
        ltas_range_list.append(ltas_range)
        ltas_slope_list.append(ltas_slope)
        f1_mean_list.append(f1_mean) 
        f1_sd_list.append(f1_sd)
        f2_mean_list.append(f2_mean)
        f2_sd_list.append(f2_sd)
        f3_mean_list.append(f3_mean)
        f3_sd_list.append(f3_sd)      
        pitch_mean_list.append(pitch_mean)
        pitch_sd_list.append(pitch_sd)
        pitch_max_list.append(pitch_max)
        pitch_min_list.append(pitch_min)
        pitch_range_list.append(pitch_range)
        intensity_mean_list.append(intensity_mean)
        intensity_sd_list.append(intensity_sd)
        intensity_max_list.append(intensity_max)
        intensity_min_list.append(intensity_min)
        intensity_range_list.append(intensity_range)
        hnr_list.append(hnr)
        duration_list.append(duration)
        localJitter_list.append(localJitter)
        localabsoluteJitter_list.append(localabsoluteJitter)
        rapJitter_list.append(rapJitter)
        ppq5Jitter_list.append(ppq5Jitter)
        ddpJitter_list.append(ddpJitter)
        localShimmer_list.append(localShimmer)
        localdbShimmer_list.append(localdbShimmer)
        apq3Shimmer_list.append(apq3Shimmer)
        aqpq5Shimmer_list.append(aqpq5Shimmer)
        apq11Shimmer_list.append(apq11Shimmer)
        ddaShimmer_list.append(ddaShimmer)            
        
    
    data = [file_list, speaker_list, gender_list, emotion_list, mfcc_list, ltas_mean_list, ltas_sd_list, ltas_max_list, ltas_min_list, ltas_range_list, ltas_slope_list, 
            f1_mean_list, f1_sd_list, f2_mean_list, f2_sd_list, f3_mean_list, f3_sd_list,
            pitch_mean_list, pitch_sd_list, pitch_max_list, pitch_min_list, pitch_range_list,
            intensity_mean_list, intensity_sd_list, intensity_max_list, intensity_min_list, intensity_range_list,
            hnr_list, duration_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list, 
            localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list]
    
    # replace Nan value by 0
    datax =[]
    for list_data in data:      
        in_arr = geek.array(list_data)
        x = geek.nan_to_num(in_arr)
        datax.append(x)
        
    df = pd.DataFrame(np.column_stack(datax), columns=['voiceID', 'speaker', 'gender', 'Emotion', 'MFCC 1', 'MFCC 2', 'MFCC 3', 'MFCC 4', 'MFCC 5', 'MFCC 6', 'MFCC 7', 'MFCC 8', 'MFCC 9', 'MFCC 10', 'MFCC 11', 'MFCC 12', 
                                                      'LTAS mean', 'LTAS stdev', 'LTAS max', 'LTAS min', 'LTAS range', 'LTAS slope', 
                                                      'Formant 1 mean', 'Formant 1 stdev', 'Formant 2 mean', 'Formant 2 stdev', 'Formant 3 mean', 'Formant 3 stdev', 
                                                      'Pitch mean', 'Pitch stdev', 'Pitch max', 'Pitch min', 'Pitch range', 
                                                      'Intensity mean', 'Intensity stdev', 'Intensity max', 'Intensity min', 'Intensity range', 
                                                      'HNR', 'Duration', 'local Jitter', 'local-absolute Jitter', 'rap Jitter', 'ppq5 Jitter', 'ddp Jitter', 
                                                      'local Shimmer', 'local-db Shimmer', 'apq3 Shimmer', 'apq5 Shimmer', 'apq11 Shimmer', 'dda Shimmer'])  #add these lists to pandas in the right order
    
    # calculate Jitter PCA 
    pcaJitterdf, jitterPCA = runPCA(df,"jitter")
    df = pd.concat([df, pcaJitterdf], axis=1)
    
    for j in jitterPCA:
        jitterpca_list.append(j[0])    
    
    # calculate Shimmer PCA     
    pcaShimmerdf, shimmerPCA = runPCA(df,"shimmer")
    df = pd.concat([df, pcaShimmerdf], axis=1)
    for s in shimmerPCA:
        shimmerpca_list.append(s[0])
    
    # Write out the updated dataframe
    #df.to_csv("EYASE (different value of pitch for each gender1).csv", index=False)   


In [125]:
tic = time.perf_counter()

extract_all_features()

toc = time.perf_counter()
print(f"Running time: {(toc - tic)/60:0.4f} minutes")

Running time: 1.1899 minutes


In [176]:
def load_data():
    x, y = [], []
    for file in glob.glob("Data/NActor_*/*.wav"): #, recursive=True
        # get the base name of the audio file
        basename = os.path.basename(file)
        # get the emotion label
        emotion = allemotion[basename.split(" ")[0].split("-")[3]]
        # get the gender label
        gender = allgender[basename.split(" ")[0].split("-")[1]]
        # get the speaker label
        speaker = basename.split(" ")[0].split("-")[2] 
            
            
        # we allow only observed_emotions we set for both gender
        if emotion not in observed_emotions:
            continue
        if gender not in observed_gender:
            continue
                
        
        # extract speech features
        result = np.array([])
        index = file_list.index(basename) #get features of each file by index
        
        file_name = file_list[index]
        #
        """
        result = np.hstack((result, mfcc_list[index])) # mfcc
        """
        #result = np.hstack((result, mel_list[index])) # mel
        
        
        result = np.hstack((result, ltas_mean_list[index])) # ltas_mean
        #result = np.hstack((result, ltas_sd_list[index])) # ltas_sd 
        #result = np.hstack((result, ltas_max_list[index])) # ltas_max
        #result = np.hstack((result, ltas_min_list[index])) # ltas_min
        #result = np.hstack((result, ltas_range_list[index])) # ltas_range
        result = np.hstack((result, ltas_slope_list[index])) # ltas_slope
        
        """
        result = np.hstack((result, f1_mean_list[index])) # f1_mean
        result = np.hstack((result, f1_sd_list[index])) # f1_sd
        result = np.hstack((result, f2_mean_list[index])) # f2_mean
        result = np.hstack((result, f2_sd_list[index])) # f2_sd
        result = np.hstack((result, f3_mean_list[index])) # f3_mean
        result = np.hstack((result, f3_sd_list[index])) # f3_sd
        """
        """
        result = np.hstack((result, pitch_mean_list[index])) # pitch_mean
        result = np.hstack((result, pitch_sd_list[index])) # pitch_sd
        result = np.hstack((result, pitch_max_list[index])) # pitch_max
        result = np.hstack((result, pitch_min_list[index])) # pitch_min
        result = np.hstack((result, pitch_range_list[index])) # pitch_range
        
        result = np.hstack((result, intensity_mean_list[index])) # intensity_mean
        result = np.hstack((result, intensity_sd_list[index])) # intensity_sd
        result = np.hstack((result, intensity_max_list[index])) # intensity_max
        result = np.hstack((result, intensity_min_list[index])) # intensity_min
        result = np.hstack((result, intensity_range_list[index])) # intensity_range

        #result = np.hstack((result, hnr_list[index])) # hnr
        #result = np.hstack((result, duration_list[index])) # duration
               
        result = np.hstack((result, localJitter_list[index])) #jitter
        result = np.hstack((result, localabsoluteJitter_list[index])) #jitter
        result = np.hstack((result, rapJitter_list[index])) #jitter
        result = np.hstack((result, ppq5Jitter_list[index])) #jitter
        result = np.hstack((result, ddpJitter_list[index])) #jitter
        result = np.hstack((result, jitterpca_list[index])) #jitter       
        
        result = np.hstack((result, localShimmer_list[index])) #shimmer
        result = np.hstack((result, localdbShimmer_list[index])) #shimmer
        result = np.hstack((result, apq3Shimmer_list[index])) #shimmer
        result = np.hstack((result, aqpq5Shimmer_list[index])) #shimmer
        result = np.hstack((result, apq11Shimmer_list[index])) #shimmer
        result = np.hstack((result, ddaShimmer_list[index])) #shimmer
        result = np.hstack((result, shimmerpca_list[index])) #shimmer
        """
        # # 
            
        # add to data
        x.append(result)
        y.append(emotion)
        
        
    # replace Nan to 0
    in_x = geek.array(x)
    
    out_x = geek.nan_to_num(in_x)
    
    return np.array(out_x), np.array(y)

In [179]:
#loading_data
x,y = load_data()

k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)

# SVM with linear kernel
model = SVC(kernel = 'linear', C = 1)

result = cross_val_score(model , x, y, cv = kf)
#print("Accuracy: {}".format(result))
print("Avg linear accuracy: {:.2f}%".format(result.mean()*100))

Avg linear accuracy: 88.89%


In [180]:
#loading_data
x,y = load_data()

k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)

# SVM with linear kernel
model = SVC(kernel = 'linear', C = 1)

result = cross_val_score(model , x, y, cv = kf)
#print("Accuracy: {}".format(result))
print("Avg linear accuracy: {:.2f}%".format(result.mean()*100))

Avg linear accuracy: 88.33%


In [165]:
#'linear', 'poly', 'rbf'
#loading_data
x,y = load_data()

k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)

# SVM with linear kernel
model = SVC(kernel = 'poly', C = 1)

result = cross_val_score(model , x, y, cv = kf)
#print("Accuracy: {}".format(result))
print("Avg poly accuracy: {:.2f}%".format(result.mean()*100))

Avg poly accuracy: 87.22%


In [166]:
#'linear', 'poly', 'rbf'
#loading_data
x,y = load_data()

k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)

# SVM with linear kernel
model = SVC(kernel = 'poly', C = 1)

result = cross_val_score(model , x, y, cv = kf)
#print("Accuracy: {}".format(result))
print("Avg poly accuracy: {:.2f}%".format(result.mean()*100))

Avg poly accuracy: 86.67%


In [169]:
#'linear', 'poly', 'rbf'
#loading_data
x,y = load_data()

k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)

# SVM with linear kernel
model = SVC(kernel = 'rbf', C = 1)

result = cross_val_score(model , x, y, cv = kf)
#print("Accuracy: {}".format(result))
print("Avg rbf accuracy: {:.2f}%".format(result.mean()*100))

Avg rbf accuracy: 85.00%


In [170]:
#'linear', 'poly', 'rbf'
#loading_data
x,y = load_data()

k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)

# SVM with linear kernel
model = SVC(kernel = 'rbf', C = 1)

result = cross_val_score(model , x, y, cv = kf)
#print("Accuracy: {}".format(result))
print("Avg rbf accuracy: {:.2f}%".format(result.mean()*100))

Avg rbf accuracy: 86.11%
